This notebook is a wrapper of https://medium.com/@erikhallstrm/hello-world-rnn-83cd7105b767. It is not supposed to be self-contained but rather used with our commentaries.



# RNN
So what is a recurent neural network? It's a neural network that models series of data such that the predictions depend of the previous step(s).

It can be used to predict data that changes over time like the prices of stock-exchange or data that needs some memory of what happened like text generation.

Let's look at the basic compenents of a RNN in the following picture :

![RNN](img/RNN.png)

The green arrows represent the input and the red arrows the output. At each (time) step of the serie, we want to predict something given a certain input. Let's say, we want to predict the price of petrol given the daily production (quantity of petrol produced). The input (green arrow) is the daily production of petrol, the output is the price of petrol that day. 

But the price of petrol also depends on what happened in the past, not just the daily production. To use that information, we put that information somehow into the state vector (blue arrow), which is recalculated at each step.

How are these state vectors computed? This is the basic equation: 

$h^{(t)} = \sigma ( W^{hx}x +W^{hh}h^{(t-1)}+b_h)$

where x is the input (green arrow), h is the vector state (blue arrow) and $\sigma$ is an activation function. In this tutorial, we use the default activation of RNN cell from tensorfow: $\tanh$.

From the state vector, we calculate the output at each step. In this tutorial, we choose a classification problem, so an activation that make sense is softmax.

$Y = softmax(W^{yh}h^{(t)}+b_y)$

where Y is the output (red arrow).

So how do we choose the size of these vectors :

input : size of the information you give at each step

output : size of the output you want to predict at each step

vector state : it's arbitrary, put what you think can encompass enough information from step to step to do the prediction

# Echo problem (our main problem tonight)
The (very basic) problem is as follows: we try to reproduce the input a few steps later. The (input and output) signal has two possibles outcomes (0 or 1). For instance: 0100100111010101

Here is a visualization of the RNN at work: 

![result](img/FinalResult.png)

The blue bar, represent the input value, the green bar the output of the RNN and the red bar, the result it's should output.

We can see that the green and the red bars match perfectly. In this example, the echo is 3 steps later.

# Backpropagation through time

The algorithm used to minimize the loss function is gradient descent. We want to use the efficient backpropagation algorithm to compute the gradients. This can be done if you "unroll the network through time" and the algorithm is called *Backpropagation through time* (BPTT). One problem with BPTT is that the computation increases with the number of (time) steps. 

Let's say you are at step 14456. Do you really need to go back all the previous 14455 steps? No. If you only consider a (small) portion of the unrolled serie and only compute the backpropagation for that portion it might be enough. This is called *Truncated Backpropagation*.

So we look at the serie in small chuncks: we only compute the backpropagation for a maximum number of steps. The next figure illustrates this.

![trunk](img/truncated_backprop.png)

The colors correspond to 0 or 1. We only use backpropagation on `truncated_backprop_lenght` steps. Once we go to the next step, we start the backprogation all over **but** we take the previous results as input.

# Batch

(hold tight for this one!). We want to do mini-batch. Oh yeah! Pause for a minute and think about how you would do that. 

You cannot take each element of the serie one by one and bundle them in a batch because... you need a serie!

One way to provide batches is to separate the serie into many smaller series and process them in parallel batch by batch. The number of smaller series is the batch size. 

If you think that this "destroys" the (time) serie a bit, you might be right but... . If the serie shows a certain continuity, you can train your RNN this way and it works! 

The next picture shows these cuts:

![batch](img/batch.png)

The whole serie is cut into 3 (`batch_size`) new series.

Let's put everything together now.

First the picture:

![together](img/together.png)

and then the explanation.

We have a serie of 36 steps. We divide it in 3 smaller series: our batch size is 3. The length of our truncated backpropagation is 4. Our batches are the dashed blocks.

**Warning:** Don't confuse the batch size and the number of batches.



# Graph model

The model is just the part in the trunkable backpropagation. It looks like this (the `truncated_backprop_length` here is 4, but we will use 15 in the code):

![model](img/modelGraph.png)

We use lists: a list of inputs, a list of outputs and a list of state vectors.

# Code implementation

In [1]:
# import the librairy we need
from __future__ import print_function, division
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [2]:
#parameter of the system
num_epochs = 100                #number of epoch
total_series_length = 50000     #lenght of the serie we look at
truncated_backprop_length = 15  #lenght of a before stop backpropagation
state_size = 4                  #size of the state vector
num_classes = 2                 #number of classes we want to predict
echo_step = 3                   #step of the echo
batch_size = 5                  #size of batch
num_batches = total_series_length//batch_size//truncated_backprop_length

In [3]:
# function that generate the fake data we will use for the this tutorial
def generateData():
    x = np.array(np.random.choice(2, total_series_length, p=[0.5, 0.5])) #choose two random value with the same probability
    y = np.roll(x, echo_step) # shif the the value of x by 3 step (that the echo part)
    y[0:echo_step] = 0    #for the first 3 step, we put 3 values (that not much important, that the begining of the data)

    #we reshape the data to put them into batch (-1 mean let the shape be calculated by itself)
    x = x.reshape((batch_size, -1))  # The first index changing slowest, subseries as rows
    y = y.reshape((batch_size, -1))

    return (x, y)

In [4]:
# the placeholder of the input and output
batchX_placeholder = tf.placeholder(tf.float32, [batch_size, truncated_backprop_length])
batchY_placeholder = tf.placeholder(tf.int32, [batch_size, truncated_backprop_length])

Remember this picture:

![together](img/together.png)

In [5]:
# this the initial state each time we begin a new trunkable backprogation line. 
# So it's like we begin series (since we don't go back in propagation), but have some non-zero intial states
init_state = tf.placeholder(tf.float32, [batch_size, state_size])

The `init_state` is represented by the curved arrows between the dashed boxes (batches).

In [6]:
# the weight and bias for the state vector space to the output space
W2 = tf.Variable(np.random.rand(state_size, num_classes),dtype=tf.float32)
b2 = tf.Variable(np.zeros((1,num_classes)), dtype=tf.float32)

We don't need to define the weight that are inside the cell (i.e. $W^{hx}$ and $W^{hh}$) because that is done automatically by tensorflow.

In [7]:
# Unpack columns
#inputs_series = tf.split(1, truncated_backprop_length, batchX_placeholder)
# tf.split -> split a tensor into multiples one and put them into a list
inputs_series = tf.split(batchX_placeholder, num_or_size_splits = truncated_backprop_length, axis=1)
# input_series is list of 15 (truncated_backprop_length) tensor of size [batch_size, 1]

#labels_series = tf.unpack(batchY_placeholder, axis=1)
# tf.unstack -> reduce the rank of the tensor along the axis choosen
labels_series = tf.unstack(batchY_placeholder, axis=1)
# labels_series is list of 15 (truncated_backprop_length) tensor of size [batch_size, ]

# Question : why do we use a different function to transform into list of tensor for x and y

We just created the list of green and red boxes:

![model](img/modelGraph.png)

In [8]:
#Let just print to see what are these
print('size of batchX_placeholder : '+str(batchX_placeholder.shape))
print('size of batchY_placeholder : '+str(batchY_placeholder.shape))

size of batchX_placeholder : (5, 15)
size of batchY_placeholder : (5, 15)


In [9]:
# the type are 
print(type(batchX_placeholder))
print(type(batchY_placeholder))
print(type(inputs_series))
print(type(labels_series))

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'list'>
<class 'list'>


In [10]:
# Let us look at the two series
inputs_series

[<tf.Tensor 'split:0' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split:1' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split:2' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split:3' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split:4' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split:5' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split:6' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split:7' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split:8' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split:9' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split:10' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split:11' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split:12' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split:13' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split:14' shape=(5, 1) dtype=float32>]

In [11]:
# Let us look at the two series
labels_series

[<tf.Tensor 'unstack:0' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:1' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:2' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:3' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:4' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:5' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:6' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:7' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:8' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:9' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:10' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:11' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:12' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:13' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:14' shape=(5,) dtype=int32>]

In [12]:
# the size of the list is 
print(len(inputs_series))
print(len(labels_series))

15
15


In [13]:
# Forward passes
# create basic cell of the RNN and define the size of the vector space
# we have not explictly tell the activation function which in the cse choose tanh. 
cell = tf.nn.rnn_cell.BasicRNNCell(state_size)

# create a rnn from the cell
# the inputs should be the list of inputs each of them should have the [batch_size, input_size]
# the initial_state size should be [batch_size, cell.state_size]
states_series, current_state = tf.contrib.rnn.static_rnn(cell = cell, inputs = inputs_series, initial_state = init_state)

# states_series is list of the state vector (i.e. a list of 15 tensor of size [5,4] )
# current_state is the final state vector of the batch (here end the the 15th states vector)
# i.e. current_state represent the curved arrow that go outside of the dashed box in previous picture

The fact that we choose the `BasicRNNCell` mean that we choose to calculate the state vector as 

$h^{(t)} = \sigma ( W^{hx}x +W^{hh}h^{(t-1)}+b_h)$

We don't have to define the matrices $W^{hx}$ and $W^{hh}$ or the vector $b_h$. This is done inside the `BasicRNNCell` and `static_rnn function`.

The fact that we use the `static_rnn` means we use the following architecture:

![RNN](img/RNN.png)

In [14]:
# calculate the logit(i.e. argument) for each element from the states_series
logits_series = [tf.matmul(state, W2) + b2 for state in states_series] #Broadcasted addition

# create the prediction for each logits_series (the red arrow)
predictions_series = [tf.nn.softmax(logits) for logits in logits_series]

The predictions here are defined by: 

$Y = softmax(W^{yh}h^{(t)}+b_y)$

where $W2$ corresponds to $W^{yh}$ and $b2$ to $b_y$.

In [15]:

# calculate the the loss for each element of the series (why sparse?)
losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels) for logits, labels in zip(logits_series,labels_series)]
# sum over the list loss associate for the different output (i.e. over the 15 elements (or truncated_backprop_length elements))
total_loss = tf.reduce_mean(losses)

In [16]:
# we use a variant of gradient descent
train_step = tf.train.AdagradOptimizer(0.3).minimize(total_loss)

In [17]:
# we define 
def plot(loss_list, predictions_series, batchX, batchY):
    plt.subplot(2, 3, 1)
    plt.cla()
    plt.plot(loss_list)

    #loop over 5 batch (here that the total number of batch)
    for batch_series_idx in range(5):
        one_hot_output_series = np.array(predictions_series)[:, batch_series_idx, :]
        
        # make a prediction from the probability (we choose the seperation at 0.5 (we could choose something depending if we care more on true postive of false negative))
        single_output_series = np.array([(1 if out[0] < 0.5 else 0) for out in one_hot_output_series])

        plt.subplot(2, 3, batch_series_idx + 2)
        plt.cla()
        plt.axis([0, truncated_backprop_length, 0, 2])
        left_offset = range(truncated_backprop_length)
        
        # we put different hight to see all the data
        plt.bar(left_offset, batchX[batch_series_idx, :], width=1, color="blue")#input
        plt.bar(left_offset, batchY[batch_series_idx, :] * 0.5, width=1, color="red")#label
        plt.bar(left_offset, single_output_series * 0.3, width=1, color="green")#calculated output

    
    clear_output(wait=True)
    plt.draw()
    plt.pause(0.0001)

In [18]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    plt.ion()
    plt.figure()
    plt.show()
    loss_list = []

    for epoch_idx in range(num_epochs):
        # we generate new data for new epoch (so it's always new data)
        x,y = generateData()
        # size of x and y : [batch_size, total_series_length/batch_size]=[5,10000]
        
        #initalize all 
        _current_state = np.zeros((batch_size, state_size))

        print("New data, epoch", epoch_idx)

        # loop over the batch (i.e. the dashed box), not to confuse with line associate with batch size (i.e. 10000/15 boxes)
        for batch_idx in range(num_batches):
            
            #indices of the begin and end of the batch
            start_idx = batch_idx * truncated_backprop_length
            end_idx = start_idx + truncated_backprop_length
            
            # the first indice is exemple indice associate in a batch
            # thus we see that take all the exemple of the batch
            batchX = x[:,start_idx:end_idx]# [5,15]
            batchY = y[:,start_idx:end_idx]# [5,15]
            
            # train and keep the different value needed
            _total_loss, _train_step, _current_state, _predictions_series = sess.run(
                [total_loss, train_step, current_state, predictions_series],
                feed_dict={
                    batchX_placeholder:batchX,# feed the input
                    batchY_placeholder:batchY,# feed the label
                    init_state:_current_state # feed the last state vector of the previous batch
                })

            loss_list.append(_total_loss)

            if batch_idx%100 == 0:
                print("Step",batch_idx, "Loss", _total_loss)
                
                plot(loss_list, _predictions_series, batchX, batchY)

plt.ioff()
plt.show()

KeyboardInterrupt: 